# Quadratic Voting

In [33]:
from torch import *
from builtins import range
import numpy as np

from torch.nn import Module, Parameter, Softmax
from torch.linalg import vector_norm
from torch.optim import SGD

In [34]:
alpha = tensor([1.0, 3.0, 2.0])
def U(x): # utility function
    return x @ alpha # this implementation supports batching

In [35]:
U(tensor([1/3, 1/3, 1/3])) 

tensor(2.)

In [36]:
U(tensor([0.0, 1.0, 0.0]))

tensor(3.)

In [37]:
U(tensor([1.0, 0.0, 0.0]))

tensor(1.)

In [38]:
batch = tensor([
    [1/3, 1/3, 1/3],
    [0.0, 1.0, 0.0],
    [1.0, 0.0, 0.0],
])

In [39]:
U(batch)

tensor([2., 3., 1.])

In [40]:
class Utility(Module): # minus utility parametrized by logits
    def __init__(self, alpha, quadratic=False):
        super().__init__()
        self.quadratic = quadratic
        self.alpha = tensor(alpha)
        self.logits = Parameter(rand(3))
        self.softmax = Softmax(dim=-1)
    def forward(self):
        x = self.x
        if self.quadratic:
            x = sqrt(x)
        return x @ self.alpha
    def get_x(self):
        return self.softmax(self.logits)
    x = property(get_x)

In [41]:
utility = Utility([1.0, 3.0, 2.0])

In [42]:
optimizer = SGD(utility.parameters(), lr=1.0)

In [43]:

# OK

utility = Utility([1.0, 3.0, 2.0])
optimizer = SGD(utility.parameters(), lr=1.0)

print(utility.x.detach().numpy())
for i in range(10_000):
    optimizer.zero_grad()
    loss = -utility()
    loss.backward()
    optimizer.step()
    if i % 1_000 == 0:
        print(utility.x.detach().numpy())

[0.33192194 0.3689132  0.2991649 ]
[0.22248855 0.49770325 0.27980822]
[1.6732406e-04 9.9949801e-01 3.3464000e-04]
[8.3509563e-05 9.9974948e-01 1.6702511e-04]
[5.5643559e-05 9.9983299e-01 1.1129248e-04]
[4.1716688e-05 9.9987483e-01 8.3437713e-05]
[3.3366323e-05 9.9989986e-01 6.6736182e-05]
[2.7801761e-05 9.9991667e-01 5.5606466e-05]
[2.3828197e-05 9.9992847e-01 4.7658963e-05]
[2.0848604e-05 9.9993742e-01 4.1699455e-05]
[1.8531469e-05 9.9994445e-01 3.7064900e-05]


In [53]:

# Here the result is wrong AFAICT, but very stably so!
# I don't get smthin prop to alpha as a result, but prop to the squares of alpha
# What does the theory says then? Yay, that's it, my bad. The effort is prop to alpha square
# and the movement of the needle prop to alpha.

utility = Utility([1.0, 3.0, 2.0], quadratic=True)
optimizer = SGD(utility.parameters(), lr=1.0)

x = utility.x.detach().numpy()
print(x, np.sqrt(x))
for i in range(10_000):
    optimizer.zero_grad()
    loss = -utility()
    loss.backward()
    optimizer.step()
    if i % 1_000 == 0:
        x = utility.x.detach().numpy()
        s = np.sqrt(x)
        print(x, s)

[0.20332044 0.35189286 0.4447866 ] [0.4509107 0.5932056 0.6669232]
[0.17262416 0.4455143  0.38186154] [0.41548064 0.66746855 0.6179495 ]
[0.07142858 0.64285713 0.2857143 ] [0.26726127 0.80178374 0.5345225 ]
[0.07142858 0.64285713 0.2857143 ] [0.26726127 0.80178374 0.5345225 ]
[0.07142858 0.64285713 0.2857143 ] [0.26726127 0.80178374 0.5345225 ]
[0.07142858 0.64285713 0.2857143 ] [0.26726127 0.80178374 0.5345225 ]
[0.07142858 0.64285713 0.2857143 ] [0.26726127 0.80178374 0.5345225 ]
[0.07142858 0.64285713 0.2857143 ] [0.26726127 0.80178374 0.5345225 ]
[0.07142858 0.64285713 0.2857143 ] [0.26726127 0.80178374 0.5345225 ]
[0.07142858 0.64285713 0.2857143 ] [0.26726127 0.80178374 0.5345225 ]
[0.07142858 0.64285713 0.2857143 ] [0.26726127 0.80178374 0.5345225 ]


In [54]:
s[1] / s[0]

2.9999998

In [55]:
s[2] / s[0]

1.9999998